# 1. Installing Dependencies

Run this file to install all dependencies needed by the Notebook. You can skip running this cell if you have runned this cell in the current environment previously.

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt
!jupyter nbextension enable --py widgetsnbextension
!jupyter serverextension enable voila --sys-prefix

Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Config option `kernel_spec_manager_class` not recognized by `EnableServerExtensionApp`.
Enabling: voila
- Writing config: /opt/conda/etc/jupyter
    - Validating...
      voila 0.2.10 OK


# 2. Getting Analysis Result

### Get the tables with highest discrepancy on write vs read frequency throughout the data warehouse
This will list down tables with the highest discrepancy on write vs read frequency.

In [5]:
import src.pipeline_analysis as pipeline_analysis
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

limited_imbalance_tables = []
def get_limited_imbalance_tables_df(limit):
    global limited_imbalance_tables
    limited_imbalance_tables_df = pipeline_analysis.get_tables_read_write_frequency_df(limit)
    limited_imbalance_tables = limited_imbalance_tables_df['Table'].tolist()
    return limited_imbalance_tables_df

limited_imbalance_tables_df = get_limited_imbalance_tables_df(5)
display(limited_imbalance_tables_df)

,Table,Read Frequency,Write Frequency
0,data-analytics-pocs.atos.temp_updates,0,12613
1,data-analytics-pocs.atos.max_timestamp_temp,0,8131
2,data-analytics-pocs.atos.identity_4m_new,0,7950
3,data-analytics-pocs._c4108c8641cb849fa23d12f19...,0,2057
4,data-analytics-pocs.public.bigquery_audit_log,0,1698


#### Get the pipeline graph data of the table
This will generate a pipeline graph file, in HTML format, under `pipeline_graph` directory. It may take sometime for this to run and generate.

In [6]:
def visualise_table_pipelines(table):
    pipeline_analysis.display_pipelines_of_table(table)

visualise_table_pipelines("data-analytics-pocs.public.bigquery_audit_log")

#### Display the pipeline graph of the table
Display the pipeline graph of the table. The thickness of the edges indicates the frequency compared to the rest of the edges in the current graph.

The result will be something like this, when you run the notebook, you will be able to click on the different nodes of the graph, each representing different tbales that are part of the pipeline of this table of interest. When you click on a node, it will display more information for this table.

![](assets/pipeline-example.gif)